In [225]:
import metpy
import metpy.calc as mpcalc
from metpy.units import units
import pandas as pd
from metpy.plots import add_metpy_logo, Hodograph, SkewT
import matplotlib.pyplot as plt
import glob
from pathlib import Path

'20240722 1200'

In [223]:
files = glob.glob("../data/soundings/valpo-humboldt/IOP1-July-22-23-2024/*")
files

['../data/soundings/valpo-humboldt/IOP1-July-22-23-2024/20240722_1200.GPS',
 '../data/soundings/valpo-humboldt/IOP1-July-22-23-2024/20240722_1600.GPS',
 '../data/soundings/valpo-humboldt/IOP1-July-22-23-2024/20240722_1900.GPS',
 '../data/soundings/valpo-humboldt/IOP1-July-22-23-2024/20240722_1730.GPS']

In [243]:
for file in files:
    df = pd.read_csv(file,
                     skiprows=[0, 1, 2, 3, 4, 5, 6, 7, 8, 10,],
                     delim_whitespace=True,)
    
    # Fix the incorrect column name due to space in Elapsed time
    good_columns = df.columns[1:]
    df = df.drop(columns="HeightE")
    df.columns = good_columns

    p = df['P'].values * units.hPa
    z = df['HeightMSL'].values * units.m
    T = df['Temp'].values * units.degC
    Td = df['Dewp'].values * units.degC
    wind_speed = df['Speed'].values * units.knots
    wind_dir = df['Dir'].values * units.degrees
    u, v = mpcalc.wind_components(wind_speed, wind_dir)

    resample_freq = 80
    fig = plt.figure(figsize=(9, 9))
    skew = SkewT(fig, rotation=45, rect=(0.1, 0.1, 0.55, 0.85))
    
    # Plot the data using normal plotting functions, in this case using
    # log scaling in Y, as dictated by the typical meteorological plot
    skew.plot(p[::resample_freq], T[::resample_freq], 'r')
    skew.plot(p[::resample_freq], Td[::resample_freq], 'g')
    skew.plot_barbs(p[::resample_freq], u[::resample_freq], v[::resample_freq])
    
    # Change to adjust data limits and give it a semblance of what we want
    skew.ax.set_adjustable('datalim')
    skew.ax.set_ylim(1000, 100)
    skew.ax.set_xlim(-20, 30)
    
    # Add the relevant special lines
    skew.plot_dry_adiabats()
    skew.plot_moist_adiabats()
    skew.plot_mixing_lines()
    time_date = Path(file).stem.replace("_", " ")
    plt.title(f"KHUM Sounding {time_date} UTC \n From Albizu Campos High School", fontsize=14)
    
    # Create a hodograph
    ax = plt.axes((0.7, 0.75, 0.2, 0.2))
    h = Hodograph(ax, component_range=15.)
    h.add_grid(increment=20)
    h.plot(u, v)
    
    
    plt.savefig(Path(file).stem + ".png")
    plt.close()